## Introduction
Electroencephalography (EEG) is a non-invasive method used to record electrical activity of the brain. EEG signals are crucial for understanding brain functions and diagnosing neurological disorders. In this project, we aim to classify different cognitive states (rest and task) using advanced deep learning techniques. The dataset used is sourced from the Mental Arithmetic Tasks Dataset available at PhysioNet.

## Objectives
The primary objective of this project is to implement and evaluate multiple deep learning models to classify EEG data. We focus on extracting meaningful features from the EEG signals and using these features to train deep learning models to distinguish between rest and task states.

## Load and Inspect the CSV File

In [7]:
import pandas as pd

# Path to the CSV file
csv_file_path = 'C:\\Users\\shail\\Downloads\\Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras-master\\subject-info.csv'

# Load the CSV file
data = pd.read_csv(csv_file_path)

# Display the first few rows of the data
print(data.head())

# Display the column names
print(data.columns)

     Subject  Age Gender  Recording year  Number of subtractions  \
0  Subject00   21      F            2011                    9.70   
1  Subject01   18      F            2011                   29.35   
2  Subject02   19      F            2012                   12.88   
3  Subject03   17      F            2010                   31.00   
4  Subject04   17      F            2010                    8.60   

   Count quality  
0              0  
1              1  
2              1  
3              1  
4              0  
Index(['Subject', 'Age', 'Gender', 'Recording year', 'Number of subtractions',
       'Count quality'],
      dtype='object')


## Modify the Script Based on the CSV Structure

The dataset consists of EEG recordings from subjects performing mental arithmetic tasks. The data is pre-cleaned and ready for use, allowing us to focus on model implementation and evaluation. Each recording is labeled to indicate whether it corresponds to a rest state (before the mental arithmetic task) or a task state (during the mental arithmetic task).

In [8]:
import os
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, AveragePooling2D, Dropout, Flatten, Dense
from tensorflow.keras.utils import to_categorical

# Load the CSV file
csv_file_path = 'C:\\Users\\shail\\Downloads\\Credit-Card-Fraud-Detection-using-Autoencoders-in-Keras-master\\subject-info.csv'
data = pd.read_csv(csv_file_path)

# Inspect the columns and data to identify EEG and state columns
print(data.head())
print(data.columns)

# Replace 'EEG_*' with actual column names for EEG signals
eeg_column_names = [col for col in data.columns if 'EEG' in col]  # Adjust this based on actual column names
eeg_data = data[eeg_column_names].values

# Identify the column that indicates rest (0) and task (1) states
# Assuming a column named 'Condition' for demonstration purposes
if 'Condition' in data.columns:
    state_column = 'Condition'  # Replace with the actual state column name

    rest_data = eeg_data[data[state_column] == 0]
    task_data = eeg_data[data[state_column] == 1]

    # Define frequency bands
    frequency_bands = {
        'Delta': (1, 4),
        'Theta': (4, 8),
        'Alpha': (8, 12),
        'Beta': (12, 30),
        'Gamma': (30, 100)
    }

    # Calculate the PSD for each band
    def calculate_psd(eeg_data, band):
        psd, freqs = welch(eeg_data, fs=256, nperseg=1024, axis=0)  # Adjust fs as per your data
        band_freqs = (freqs >= band[0]) & (freqs <= band[1])
        psd_band = np.mean(psd[band_freqs], axis=0)
        return psd_band, freqs[band_freqs]

    psd_rest, psd_task = {}, {}

    for band_name, band in frequency_bands.items():
        psd_rest[band_name], _ = calculate_psd(rest_data, band)
        psd_task[band_name], _ = calculate_psd(task_data, band)

    # Plot the PSDs
    def plot_psd(psd_rest, psd_task, band_name):
        plt.figure(figsize=(10, 5))
        plt.plot(np.mean(psd_rest[band_name], axis=0), label='Rest')
        plt.plot(np.mean(psd_task[band_name], axis=0), label='Task')
        plt.title(f'PSD Comparison for {band_name} Band')
        plt.xlabel('Frequency (Hz)')
        plt.ylabel('Power Spectral Density (dB/Hz)')
        plt.legend()
        plt.show()

    for band_name in frequency_bands.keys():
        plot_psd(psd_rest, psd_task, band_name)

    # Feature Extraction and Deep Learning Model Training
    def extract_features(psd_dict):
        features = []
        for band_name, psd in psd_dict.items():
            features.append(np.mean(psd, axis=0))
        return np.array(features).flatten()

    features_rest = extract_features(psd_rest)
    features_task = extract_features(psd_task)

    X = np.vstack((features_rest, features_task))
    y = np.array([0, 1])  # 0 for rest, 1 for task

    # Standardize the features
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Reshape data for deep learning models
    X = X.reshape(X.shape[0], X.shape[1], 1, 1)  # Adjust dimensions as needed

    # Create EEGNet model
    def create_eegnet_model(input_shape):
        model = Sequential()
        model.add(Conv2D(16, (1, 64), padding='same', input_shape=input_shape))
        model.add(BatchNormalization())
        model.add(Activation('elu'))
        model.add(AveragePooling2D((1, 4)))
        model.add(Dropout(0.25))

        model.add(Conv2D(4, (1, 16), padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('elu'))
        model.add(AveragePooling2D((1, 8)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(2, activation='softmax'))
        return model

    eegnet_model = create_eegnet_model(X.shape[1:])
    eegnet_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # Convert labels to categorical
    y_categorical = to_categorical(y)

    # Train the model
    eegnet_model.fit(X, y_categorical, epochs=10, batch_size=2)

    # Evaluate the model
    def evaluate_model(model, X, y):
        kf = KFold(n_splits=5)
        accuracies, precisions, recalls, f1s = [], [], []

        for train_index, test_index in kf.split(X):
            X_train, X_test = X[train_index], X[test_index]
            y_train, y_test = y_categorical[train_index], y_categorical[test_index]

            model.fit(X_train, y_train, epochs=10, batch_size=2, verbose=0)
            y_pred = np.argmax(model.predict(X_test), axis=1)
            y_true = np.argmax(y_test, axis=1)

            accuracies.append(accuracy_score(y_true, y_pred))
            precisions.append(precision_score(y_true, y_pred))
            recalls.append(recall_score(y_true, y_pred))
            f1s.append(f1_score(y_true, y_pred))

        print(f'Accuracy: {np.mean(accuracies)}')
        print(f'Precision: {np.mean(precisions)}')
        print(f'Recall: {np.mean(recalls)}')
        print(f'F1-score: {np.mean(f1s)}')

    evaluate_model(eegnet_model, X, y)
else:
    print("State column not found. Please identify the correct column indicating rest and task states.")

     Subject  Age Gender  Recording year  Number of subtractions  \
0  Subject00   21      F            2011                    9.70   
1  Subject01   18      F            2011                   29.35   
2  Subject02   19      F            2012                   12.88   
3  Subject03   17      F            2010                   31.00   
4  Subject04   17      F            2010                    8.60   

   Count quality  
0              0  
1              1  
2              1  
3              1  
4              0  
Index(['Subject', 'Age', 'Gender', 'Recording year', 'Number of subtractions',
       'Count quality'],
      dtype='object')
State column not found. Please identify the correct column indicating rest and task states.


## Deep Learning Models:

EEGNet: A compact convolutional neural network designed specifically for EEG signal classification. It efficiently captures spatial and temporal features from EEG data.
Other Models: Depending on the choice, additional models such as TSCeption (for time-series classification), Vision Transformer (ViT), ATCNet, or Variational Autoencoder (VAE) can be used to explore different deep learning architectures and their effectiveness in EEG classification.
Implementation and Evaluation
The implementation involves preprocessing the data, extracting features, and training the models. The evaluation metrics used are accuracy, precision, recall, and F1-score. These metrics provide a comprehensive understanding of the model's performance in distinguishing between rest and task states.

## Conclusion
This project demonstrates the application of deep learning techniques to classify EEG data, showcasing the potential of these methods in analyzing brain activity and cognitive states. The insights gained from PSD analysis and model evaluation contribute to our understanding of how different brain states can be identified and classified using EEG signals.